First I use LD-pruned SNPs (from 001_DF_pooled_data_explore.ipynb) to estimate the covariance matrix across 5 independent runs of the core mode of baypass, and to ensure convergence I calculate the pairwise correlation of the matrices across runs (I found that these runs were highly correlated, r2 > 0.998).

Then I run baypass in STD mode (mcmc) for SNPs in our data.

In [6]:
from pythonimports import *

DIR = '/data/projects/pool_seq/DF_datasets'
snpdir = op.join(DIR, 'DF_pooled_GEA/DF_pooled/snpsANDindels/03_maf-p01_RD-recalculated')
assert op.exists(snpdir)
baydir = makedir(op.join(snpdir, 'baypass'))
op.exists(baydir)

True

# create infiles to baypass to estimate neutral structure

#### <center> genotyping data

"The genotyping data file is simply organized as a matrix with
nsnp rows and 2 ∗ npop columns. The row field separator is a space. More
precisely, each row corresponds to one marker and the number of columns is
twice the number of populations because each pair of numbers corresponds to
each allele (or read counts for PoolSeq experiment) counts in one population"

In [21]:
baydir

'/data/projects/pool_seq/DF_datasets/DF_pooled_GEA/DF_pooled/snpsANDindels/03_maf-p01_RD-recalculated/baypass'

In [46]:
fs(baydir, bnames=True, dirs=False)

['baseline-missing-data_20-dp-1000_random-snps_1-per-contig-gt1Kbp_r2-lessthan-p20.pkl',
 'baseline-missing-data_20-dp-1000_random-snps_1-per-contig-gt1Kbp_r2-lessthan-p20_table.txt']

In [51]:
prunedloci = pklload(op.join(baydir, 'baseline-missing-data_20-dp-1000_random-snps_1-per-contig-gt1Kbp_r2-lessthan-p20.pkl'))
len(prunedloci)

3591

In [52]:
# load snps that are LD-pruned
# these loci were chosen in 001_DF_pooled_data_explore.ipynb
reduced = pd.read_table(op.join(baydir, 'baseline-missing-data_20-dp-1000_random-snps_1-per-contig-gt1Kbp_r2-lessthan-p20_table.txt'))
reduced.index = reduced['locus'].tolist()
print(reduced.shape)
reduced.head()

(3591, 711)


,CHROM,POS,REF,ALT,AF,QUAL,TYPE,FILTER,ADP,WT,HET,HOM,NC,DF_p1.GT,DF_p1.GQ,DF_p1.SDP,DF_p1.DP,DF_p1.FREQ,DF_p1.PVAL,DF_p1.AD,DF_p1.RD,DF_p18.GT,DF_p18.GQ,DF_p18.SDP,DF_p18.DP,DF_p18.FREQ,DF_p18.PVAL,DF_p18.AD,DF_p18.RD,DF_p19.GT,DF_p19.GQ,DF_p19.SDP,DF_p19.DP,DF_p19.FREQ,DF_p19.PVAL,DF_p19.AD,DF_p19.RD,DF_p2.GT,DF_p2.GQ,DF_p2.SDP,DF_p2.DP,DF_p2.FREQ,DF_p2.PVAL,DF_p2.AD,DF_p2.RD,DF_p20.GT,DF_p20.GQ,DF_p20.SDP,DF_p20.DP,DF_p20.FREQ,...,DF_p94.GT,DF_p94.GQ,DF_p94.SDP,DF_p94.DP,DF_p94.FREQ,DF_p94.PVAL,DF_p94.AD,DF_p94.RD,DF_p95.GT,DF_p95.GQ,DF_p95.SDP,DF_p95.DP,DF_p95.FREQ,DF_p95.PVAL,DF_p95.AD,DF_p95.RD,DF_p96.GT,DF_p96.GQ,DF_p96.SDP,DF_p96.DP,DF_p96.FREQ,DF_p96.PVAL,DF_p96.AD,DF_p96.RD,DF_p97.GT,DF_p97.GQ,DF_p97.SDP,DF_p97.DP,DF_p97.FREQ,DF_p97.PVAL,DF_p97.AD,DF_p97.RD,DF_p98.GT,DF_p98.GQ,DF_p98.SDP,DF_p98.DP,DF_p98.FREQ,DF_p98.PVAL,DF_p98.AD,DF_p98.RD,DF_p17.GT,DF_p17.GQ,DF_p17.SDP,DF_p17.DP,DF_p17.FREQ,DF_p17.PVAL,DF_p17.AD,DF_p17.RD,locus,MAF
jcf7190000127863-64936,jcf7190000127863,64936,A,G,0.828873,-10.0,SNP,PASS,72,0,27,60,0,G/G,255.0,84,84.0,84.52%,2.026700e-34,71.0,13.0,A/G,205.0,59,59.0,77.97%,2.911200e-21,46.0,13.0,G/G,255.0,95,95.0,86.32%,2.270000e-40,82.0,13.0,G/G,110.0,29,29.0,82.76%,9.254400e-12,24.0,5.0,A/G,130.0,49,49.0,65.31%,...,G/G,255.0,55,55.0,94.55%,3.131700e-28,52.0,3.0,G/G,255.0,92,92.0,85.87%,9.693500e-39,79.0,13.0,A/G,255.0,79,79.0,75.95%,3.740900e-27,60.0,19.0,G/G,255.0,96,96.0,83.33%,2.636200e-38,80.0,16.0,G/G,255.0,81,81.0,90.12%,1.930100e-37,73.0,8.0,A/G,110.0,36,36.0,72.22%,9.211800e-12,26.0,10.0,jcf7190000127863-64936,0.171127
jcf7190000135978-38057,jcf7190000135978,38057,T,A,0.011262,-10.0,SNP,PASS,54,85,2,0,0,T/T,66.0,42,42.0,2.38%,5.000000e-01,1.0,41.0,T/T,111.0,60,60.0,0%,1.000000e+00,0.0,60.0,T/T,167.0,98,98.0,1.02%,5.000000e-01,1.0,97.0,T/T,25.0,21,21.0,9.52%,2.439000e-01,2.0,19.0,T/T,74.0,40,40.0,0%,...,T/T,96.0,52,52.0,0%,1.000000e+00,0.0,52.0,T/T,100.0,54,54.0,0%,1.000000e+00,0.0,54.0,T/T,97.0,51,51.0,0%,1.000000e+00,0.0,51.0,T/T,97.0,51,51.0,0%,1.000000e+00,0.0,51.0,T/T,77.0,54,54.0,3.7%,2.476600e-01,2.0,52.0,T/T,44.0,30,30.0,3.33%,5.000000e-01,1.0,29.0,jcf7190000135978-38057,0.011262
jcf7190000149428-167816,jcf7190000149428,167816,T,C,0.032015,-10.0,SNP,PASS,95,60,27,0,0,T/T,295.0,157,157.0,0%,1.000000e+00,0.0,157.0,T/C,41.0,69,69.0,18.84%,6.550100e-05,13.0,56.0,T/C,48.0,78,78.0,19.23%,1.451200e-05,15.0,63.0,T/T,59.0,32,32.0,0%,1.000000e+00,0.0,32.0,T/C,19.0,30,30.0,NaN,...,T/T,130.0,83,83.0,2.41%,2.484800e-01,2.0,81.0,T/T,336.0,179,179.0,0%,1.000000e+00,0.0,179.0,T/T,197.0,106,106.0,0%,1.000000e+00,0.0,106.0,T/T,235.0,125,125.0,0%,1.000000e+00,0.0,125.0,T/T,209.0,112,112.0,0%,1.000000e+00,0.0,112.0,T/T,66.0,36,36.0,0%,1.000000e+00,0.0,36.0,jcf7190000149428-167816,0.032015
jcf7190000033887-69342,jcf7190000033887,69342,T,C,0.112383,-10.0,SNP,PASS,82,22,65,0,0,T/C,27.0,107,107.0,8.41%,1.639200e-03,9.0,98.0,T/C,31.0,86,86.0,11.63%,7.402300e-04,10.0,76.0,T/C,27.0,114,114.0,7.89%,1.657600e-03,9.0,105.0,T/T,25.0,25,25.0,12%,1.173500e-01,3.0,22.0,T/T,62.0,40,40.0,2.5%,...,T/T,164.0,88,88.0,0%,1.000000e+00,0.0,88.0,T/C,40.0,113,113.0,11.5%,8.469500e-05,13.0,100.0,T/C,41.0,92,92.0,14.13%,7.743100e-05,13.0,79.0,T/C,54.0,101,101.0,16.83%,3.661600e-06,17.0,84.0,T/T,117.0,81,81.0,3.7%,1.226700e-01,3.0,78.0,T/C,15.0,29,29.0,NaN,2.591700e-02,5.0,24.0,jcf7190000033887-69342,0.112383
jcf7190000139702-166207,jcf7190000139702,166207,G,A,0.011277,-10.0,SNP,PASS,394,58,29,0,0,G/A,95.0,558,558.0,5.56%,3.033900e-10,31.0,527.0,G/G,543.0,289,289.0,0%,1.000000e+00,0.0,289.0,G/G,640.0,342,342.0,0%,1.000000e+00,0.0,342.0,G/A,15.0,74,74.0,NaN,2.914600e-02,5.0,69.0,G/G,175.0,93,93.0,0%,...,G/G,614.0,327,327.0,0%,1.000000e+00,0.0,327.0,G/G,1438.0,768,768.0,0%,1.000000e+00,0.0,768.0,G/A,24.0,499,499.0,1.6%,3.797400e-03,8.0,491.0,G/A,45.0,554,554.0,2.71%,2.772300e-05,15.0,539.0,G/G,700.0,374,374.0,0%,1.000000e+00,0.0,374.0,G/G,258.0,162,162.0,1.85%,1.238400e-01,3.0,159.0,jcf7190000139702-166207,0.011277


In [54]:
len(set(prunedloci).intersection(reduced.index)) == len(prunedloci)

True

In [56]:
def get_counts(loci, numpops=0):
    """Get read counts for global major and minor allele."""
    import pandas
    from collections import OrderedDict
    import tqdm
    
    pops = [col.replace(".FREQ","") for col in reduced.columns if '.FREQ' in col]
    assert len(pops) == numpops
    read_counts = OrderedDict()
    for locus in tqdm.tqdm_notebook(loci):
        if reduced.loc[locus, 'AF'] > 0.5:
            # this matches calculation of MAF from 001_JP_pooled_pangenome_data_explore.ipynb
            refismajor = False
            majortag = 'AD'
            minortag = 'RD'
        else:
            refismajor = True
            majortag = 'RD'
            minortag = 'AD'
        if refismajor is True:
#             print('checking ', locus)
            # double check that it matches MAF calculation
            assert reduced.loc[locus, 'AF'] == reduced.loc[locus, 'MAF']
        for pop in pops:
            for which,tag in zip(['major','minor'],[majortag, minortag]):
                newcol = "%s-%s" % (pop,which)
                if newcol not in read_counts:
                    read_counts[newcol] = OrderedDict()
                try:
                    read_counts[newcol][locus] = int(reduced.loc[locus, "%s.%s" % (pop, tag)])
                except ValueError as e:
                    # missing data
                    read_counts[newcol][locus] = 0
    return pandas.DataFrame(read_counts)

In [57]:
# get the gfile using the prunedloci
neutral_read_counts = get_counts(prunedloci, numpops=87)
neutral_read_counts.head()

/data/programs/brandon_anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


,DF_p1-major,DF_p1-minor,DF_p18-major,DF_p18-minor,DF_p19-major,DF_p19-minor,DF_p2-major,DF_p2-minor,DF_p20-major,DF_p20-minor,DF_p23-major,DF_p23-minor,DF_p24-major,DF_p24-minor,DF_p25-major,DF_p25-minor,DF_p26-major,DF_p26-minor,DF_p27-major,DF_p27-minor,DF_p28-major,DF_p28-minor,DF_p10-major,DF_p10-minor,DF_p29-major,DF_p29-minor,DF_p3-major,DF_p3-minor,DF_p30-major,DF_p30-minor,DF_p31-major,DF_p31-minor,DF_p32-major,DF_p32-minor,DF_p33-major,DF_p33-minor,DF_p34-major,DF_p34-minor,DF_p35-major,DF_p35-minor,DF_p36-major,DF_p36-minor,DF_p37-major,DF_p37-minor,DF_p11-major,DF_p11-minor,DF_p38-major,DF_p38-minor,DF_p39-major,DF_p39-minor,...,DF_p79-major,DF_p79-minor,DF_p8-major,DF_p8-minor,DF_p80-major,DF_p80-minor,DF_p81-major,DF_p81-minor,DF_p15-major,DF_p15-minor,DF_p82-major,DF_p82-minor,DF_p83-major,DF_p83-minor,DF_p84-major,DF_p84-minor,DF_p85-major,DF_p85-minor,DF_p86-major,DF_p86-minor,DF_p87-major,DF_p87-minor,DF_p88-major,DF_p88-minor,DF_p89-major,DF_p89-minor,DF_p9-major,DF_p9-minor,DF_p90-major,DF_p90-minor,DF_p16-major,DF_p16-minor,DF_p91-major,DF_p91-minor,DF_p92-major,DF_p92-minor,DF_p93-major,DF_p93-minor,DF_p94-major,DF_p94-minor,DF_p95-major,DF_p95-minor,DF_p96-major,DF_p96-minor,DF_p97-major,DF_p97-minor,DF_p98-major,DF_p98-minor,DF_p17-major,DF_p17-minor
jcf7190000127863-64936,71,13,46,13,82,13,24,5,32,17,55,11,50,10,68,20,69,14,64,4,69,12,59,9,68,5,56,7,79,24,63,5,56,18,67,6,61,13,83,4,57,11,29,11,62,13,58,7,56,5,...,56,15,72,2,71,9,68,10,60,14,70,13,79,20,39,10,51,12,44,16,82,9,56,5,48,15,63,10,70,10,70,12,57,21,61,6,51,16,52,3,79,13,60,19,80,16,73,8,26,10
jcf7190000135978-38057,41,1,60,0,97,1,19,2,40,0,31,1,49,7,52,0,39,0,48,0,46,0,48,2,74,0,68,1,62,0,53,1,50,1,62,0,71,0,77,1,75,0,20,0,41,6,45,0,52,0,...,43,0,47,0,56,0,48,0,55,0,49,0,77,2,42,0,55,0,39,0,64,0,63,1,51,0,55,1,53,0,63,0,62,1,50,0,48,0,52,0,54,0,51,0,51,0,52,2,29,1
jcf7190000149428-167816,157,0,56,13,63,15,32,0,24,6,108,0,175,0,115,0,116,0,91,0,111,0,115,0,88,0,65,16,136,0,86,0,76,0,86,6,77,6,71,9,69,3,57,1,110,0,56,4,59,8,...,131,0,50,5,167,0,160,0,72,0,138,0,136,0,49,0,92,7,90,9,73,7,65,5,60,1,50,1,71,0,151,0,123,0,99,0,77,0,81,2,179,0,106,0,125,0,112,0,36,0
jcf7190000033887-69342,98,9,76,10,105,9,22,3,39,1,58,4,80,14,90,7,70,9,74,12,78,4,56,12,63,16,67,8,95,11,61,6,58,9,70,8,94,1,97,15,70,7,50,0,73,5,52,21,62,19,...,84,7,53,8,110,16,68,24,53,4,89,15,83,6,42,0,78,19,72,13,78,10,75,7,60,9,63,6,59,4,91,17,97,9,52,6,71,3,88,0,100,13,79,13,84,17,78,3,24,5
jcf7190000139702-166207,527,31,289,0,342,0,69,5,93,0,346,20,540,3,338,12,375,11,414,11,319,15,373,11,296,3,267,0,427,4,323,2,259,8,406,1,362,0,488,0,369,4,223,0,395,9,269,0,293,2,...,498,5,317,0,664,15,573,11,365,2,542,9,643,5,196,1,351,5,444,0,459,1,336,2,266,0,358,1,227,6,620,3,447,14,294,9,390,7,327,0,768,0,491,8,539,15,374,0,159,3


In [59]:
# assert that DP = RD + AD
dpcols = [col for col in reduced.columns if '.DP' in col]
for dpcol in dpcols:
    pop = dpcol.split(".")[0]
    adcol = f"{pop}.AD"
    rdcol = f"{pop}.RD"
    assert all(reduced[dpcol] == (reduced[adcol]+reduced[rdcol]))

In [60]:
baydir

'/data/projects/pool_seq/DF_datasets/DF_pooled_GEA/DF_pooled/snpsANDindels/03_maf-p01_RD-recalculated/baypass'

In [61]:
# save
neutral_read_counts.to_csv(op.join(baydir, 'neutral_gfile_HEADERIDX.txt'), sep='\t', index=True)
neutral_read_counts.to_csv(op.join(baydir, 'neutral_gfile_noheaderidx.txt'), sep='\t', index=False, header=False)

#### <center> poolsizefile

"For Pool–Seq experiment, the haploid size (twice the number of pooled in- dividuals for diploid species) of each population should be provided. ... The order of the populations in the pool size file must be the same as in the allele count (and the covariate) data file(s)."

In [62]:
DIR

'/data/projects/pool_seq/DF_datasets'

In [65]:
# the reason the ploidy.keys() are not in same order of table ...
# ... is because varscan puts Sample1.X then Sample10.X ... (JP_p31 is the 10th file into varscan)
ploidy = pklload(op.join(DIR, 'DF_pooled_GEA/DF_pooled/pkl_files/ploidy.pkl'))['DF_pooled']
count = 0
for k,v in ploidy.items():
    print(k,v)
    count += 1
    if count == 5:
        break

DF_p1 80
DF_p10 80
DF_p11 80
DF_p12 80
DF_p13 80


In [66]:
# get ploidy of pops in same order as in gfile
pops = [col.replace("-major","") for col in neutral_read_counts.columns if '-major' in col]
vals = OrderedDict()
for pop in pops:
    vals[pop] = OrderedDict()
    vals[pop][0] = ploidy[pop]
poolsizefile = pd.DataFrame(vals)
poolsizefile

,DF_p1,DF_p18,DF_p19,DF_p2,DF_p20,DF_p23,DF_p24,DF_p25,DF_p26,DF_p27,DF_p28,DF_p10,DF_p29,DF_p3,DF_p30,DF_p31,DF_p32,DF_p33,DF_p34,DF_p35,DF_p36,DF_p37,DF_p11,DF_p38,DF_p39,DF_p4,DF_p40,DF_p41,DF_p42,DF_p43,DF_p44,DF_p45,DF_p46,DF_p12,DF_p47,DF_p48,DF_p49,DF_p5,DF_p50,DF_p51,DF_p52,DF_p53,DF_p54,DF_p55,DF_p13,DF_p56,DF_p57,DF_p58,DF_p59,DF_p6,DF_p60,DF_p61,DF_p62,DF_p7,DF_p72,DF_p14,DF_p73,DF_p74,DF_p75,DF_p76,DF_p77,DF_p78,DF_p79,DF_p8,DF_p80,DF_p81,DF_p15,DF_p82,DF_p83,DF_p84,DF_p85,DF_p86,DF_p87,DF_p88,DF_p89,DF_p9,DF_p90,DF_p16,DF_p91,DF_p92,DF_p93,DF_p94,DF_p95,DF_p96,DF_p97,DF_p98,DF_p17
0,80,80,80,80,80,80,80,80,76,80,80,80,78,80,80,80,80,80,78,80,80,74,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,70,80,80,80,80,76,74,80,80,78,80,80,80,80,80,80,80,80,80,80,80,80,66,80,80,78,80,78,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80


In [67]:
neutral_read_counts.head()

,DF_p1-major,DF_p1-minor,DF_p18-major,DF_p18-minor,DF_p19-major,DF_p19-minor,DF_p2-major,DF_p2-minor,DF_p20-major,DF_p20-minor,DF_p23-major,DF_p23-minor,DF_p24-major,DF_p24-minor,DF_p25-major,DF_p25-minor,DF_p26-major,DF_p26-minor,DF_p27-major,DF_p27-minor,DF_p28-major,DF_p28-minor,DF_p10-major,DF_p10-minor,DF_p29-major,DF_p29-minor,DF_p3-major,DF_p3-minor,DF_p30-major,DF_p30-minor,DF_p31-major,DF_p31-minor,DF_p32-major,DF_p32-minor,DF_p33-major,DF_p33-minor,DF_p34-major,DF_p34-minor,DF_p35-major,DF_p35-minor,DF_p36-major,DF_p36-minor,DF_p37-major,DF_p37-minor,DF_p11-major,DF_p11-minor,DF_p38-major,DF_p38-minor,DF_p39-major,DF_p39-minor,...,DF_p79-major,DF_p79-minor,DF_p8-major,DF_p8-minor,DF_p80-major,DF_p80-minor,DF_p81-major,DF_p81-minor,DF_p15-major,DF_p15-minor,DF_p82-major,DF_p82-minor,DF_p83-major,DF_p83-minor,DF_p84-major,DF_p84-minor,DF_p85-major,DF_p85-minor,DF_p86-major,DF_p86-minor,DF_p87-major,DF_p87-minor,DF_p88-major,DF_p88-minor,DF_p89-major,DF_p89-minor,DF_p9-major,DF_p9-minor,DF_p90-major,DF_p90-minor,DF_p16-major,DF_p16-minor,DF_p91-major,DF_p91-minor,DF_p92-major,DF_p92-minor,DF_p93-major,DF_p93-minor,DF_p94-major,DF_p94-minor,DF_p95-major,DF_p95-minor,DF_p96-major,DF_p96-minor,DF_p97-major,DF_p97-minor,DF_p98-major,DF_p98-minor,DF_p17-major,DF_p17-minor
jcf7190000127863-64936,71,13,46,13,82,13,24,5,32,17,55,11,50,10,68,20,69,14,64,4,69,12,59,9,68,5,56,7,79,24,63,5,56,18,67,6,61,13,83,4,57,11,29,11,62,13,58,7,56,5,...,56,15,72,2,71,9,68,10,60,14,70,13,79,20,39,10,51,12,44,16,82,9,56,5,48,15,63,10,70,10,70,12,57,21,61,6,51,16,52,3,79,13,60,19,80,16,73,8,26,10
jcf7190000135978-38057,41,1,60,0,97,1,19,2,40,0,31,1,49,7,52,0,39,0,48,0,46,0,48,2,74,0,68,1,62,0,53,1,50,1,62,0,71,0,77,1,75,0,20,0,41,6,45,0,52,0,...,43,0,47,0,56,0,48,0,55,0,49,0,77,2,42,0,55,0,39,0,64,0,63,1,51,0,55,1,53,0,63,0,62,1,50,0,48,0,52,0,54,0,51,0,51,0,52,2,29,1
jcf7190000149428-167816,157,0,56,13,63,15,32,0,24,6,108,0,175,0,115,0,116,0,91,0,111,0,115,0,88,0,65,16,136,0,86,0,76,0,86,6,77,6,71,9,69,3,57,1,110,0,56,4,59,8,...,131,0,50,5,167,0,160,0,72,0,138,0,136,0,49,0,92,7,90,9,73,7,65,5,60,1,50,1,71,0,151,0,123,0,99,0,77,0,81,2,179,0,106,0,125,0,112,0,36,0
jcf7190000033887-69342,98,9,76,10,105,9,22,3,39,1,58,4,80,14,90,7,70,9,74,12,78,4,56,12,63,16,67,8,95,11,61,6,58,9,70,8,94,1,97,15,70,7,50,0,73,5,52,21,62,19,...,84,7,53,8,110,16,68,24,53,4,89,15,83,6,42,0,78,19,72,13,78,10,75,7,60,9,63,6,59,4,91,17,97,9,52,6,71,3,88,0,100,13,79,13,84,17,78,3,24,5
jcf7190000139702-166207,527,31,289,0,342,0,69,5,93,0,346,20,540,3,338,12,375,11,414,11,319,15,373,11,296,3,267,0,427,4,323,2,259,8,406,1,362,0,488,0,369,4,223,0,395,9,269,0,293,2,...,498,5,317,0,664,15,573,11,365,2,542,9,643,5,196,1,351,5,444,0,459,1,336,2,266,0,358,1,227,6,620,3,447,14,294,9,390,7,327,0,768,0,491,8,539,15,374,0,159,3


In [68]:
# save, no need to name as 'neutral'
poolsizefile.to_csv(op.join(baydir, 'poolsizefile_HEADERIDX.txt'), sep='\t', index=False)
poolsizefile.to_csv(op.join(baydir, 'poolsizefile_noheaderidx.txt'), sep='\t', index=False, header=False)

#### <center> create random runs

#### estimate covariance matrix (run in bash)
```bash
cd /lu213/brandon.lind/projects/pool_seq/DF_datasets/DF_pooled_GEA/DF_pooled/snpsANDindels/03_maf-p01_RD-recalculated/baypass
mkdir neutral_runs
cd neutral_runs

export gfile="/lu213/brandon.lind/projects/pool_seq/DF_datasets/DF_pooled_GEA/DF_pooled/snpsANDindels/03_maf-p01_RD-recalculated/baypass/neutral_gfile_noheaderidx.txt"

export poolsizefile="/lu213/brandon.lind/projects/pool_seq/DF_datasets/DF_pooled_GEA/DF_pooled/snpsANDindels/03_maf-p01_RD-recalculated/baypass/poolsizefile_noheaderidx.txt"



# These commands took about 7 hours each to complete


# chain 1
/data/programs/baypass_2.2/sources/g_baypass -gfile $gfile -poolsizefile $poolsizefile \
-nthreads 8 -seed $seed -print_omega_samples -outprefix chain_1 > chain_1_stdout.txt &

# chain 2
/data/programs/baypass_2.2/sources/g_baypass -gfile $gfile -poolsizefile $poolsizefile \
-nthreads 8 -seed $seed -print_omega_samples -outprefix chain_2 > chain_2_stdout.txt &

# chain 3
/data/programs/baypass_2.2/sources/g_baypass -gfile $gfile -poolsizefile $poolsizefile \
-nthreads 8 -seed $seed -print_omega_samples -outprefix chain_3 > chain_3_stdout.txt &

# chain 4
/data/programs/baypass_2.2/sources/g_baypass -gfile $gfile -poolsizefile $poolsizefile \
-nthreads 8 -seed $seed -print_omega_samples -outprefix chain_4 > chain_4_stdout.txt &

# chain 5
/data/programs/baypass_2.2/sources/g_baypass -gfile $gfile -poolsizefile $poolsizefile \
-nthreads 8 -seed $seed -print_omega_samples -outprefix chain_5 > chain_5_stdout.txt &

```